## Credit Risk Analysis Report
#### To receive all points, you must:

    The purpose of this analysis is to identify whether or not a loan would be healthy or high-risk based on the borrower.

    Both models performed with high accuracy, but the OverSampler model is superior because it identifies healthy and high-risk loans with 99% accuracy, whereas the imbalanced model performed with 100% accuracy when identifying healthy loans and 87% accuracy when identifying high-risk loans. 
    
    Imbalanced Logistic Regression Model:
        Balanced Accuracy: 94.4%
        Precision: 100% (0-'healthy loans'), 87% (1-'high-risk loans')
        Recall: 100% (0-'healthy loans'), 89% (1-'high-risk loans')
    
    RandomOverSampler Logistic Regression Model:
        Balanced Accuracy: 99.4%
        Precision: 99% (0-'healthy loans'), 99% (1-'high-risk loans')
        Recall: 99% (0-'healthy loans'), 99% (1-'high-risk loans')

    The increased accuracy of the OverSampler model makes it the preferable choice because it better mitigates any risk involved with lending. While the imbalanced model still performed well, in the Finance and Banking industry the difference between 87% accuracy and 99% accuracy can be stark. What's more, this difference in accuracy arises with identifying high-risk loans, meaning that the imbalanced model is more likely to misidentify a high-risk loan as a healthy one, introducing unnecessary risk to the institution.


In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [3]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv('Resources/lending_data.csv')

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
x = lending_data_df.drop(columns=['loan_status'])

In [5]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [6]:
# Review the X variable DataFrame
x.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [7]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(random_state=1)
classifier

# Fit the model using training data
log_reg_model = classifier.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

In [12]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18679,    80],
       [   67,   558]], dtype=int64)

In [13]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
print(classification_report(y_test, predictions, target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts the healthy loan with 100% accuracy and high-risk loan with 87% accuracy. Both can be considered good accuracy rates. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = oversampler.fit_resample(X_train, y_train)

In [16]:
# Count the distinct values of the resampled labels data
y_res.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [17]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(random_state=1)
classifier

# Fit the model using the resampled training data
log_reg_model = classifier.fit(X_res, y_res)

# Make a prediction using the testing data
predictions = classifier.predict(X_res)
pd.DataFrame({"Prediction": predictions, "Actual": y_res})

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
112549,1,1
112550,1,1
112551,1,1
112552,1,1


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_res, predictions)

0.994180571103648

In [20]:
# Generate a confusion matrix for the model
confusion_matrix(y_res, predictions)

array([[55945,   332],
       [  323, 55954]], dtype=int64)

In [19]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
print(classification_report(y_res, predictions, target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       0.99      0.99      0.99     56277
High-Risk Loan       0.99      0.99      0.99     56277

      accuracy                           0.99    112554
     macro avg       0.99      0.99      0.99    112554
  weighted avg       0.99      0.99      0.99    112554



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Using the RandomOverSampler greatly improved the logistic regression model's ability to predict whether an item is a healthy loan or high-risk loan. The change in healthy loan prediction from 100% to 99% is acceptable, as predicting high-risk loans at 99% instead of 87% is much more preferable. 